In [ ]:
# Copyright (c) Microsoft. All rights reserved.
# Licensed under the MIT license. See LICENSE file in the project root for full license information.

/!pip install azure-iot-device
/!pip install azure-iot-hub

import random
import random
import time
import os
from azure.iot.device import IoTHubDeviceClient, Message, MethodResponse

# The device connection string to authenticate the device with your IoT hub.
# Using the Azure CLI:
# az iot hub device-identity show-connection-string --hub-name {YourIoTHubName} --device-id MyNodeDevice --output table


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 164 kB 29.1 MB/s 
     |████████████████████████████████| 99 kB 9.4 MB/s 
     |████████████████████████████████| 139 kB 48.8 MB/s 
     |████████████████████████████████| 62 kB 1.9 MB/s 
  Created wheel for paho-mqtt: filename=paho_mqtt-1.6.1-py3-none-any.whl size=62133 sha256=18395e7bca9452488e6a72f1bac4ee10397d7c2f92d7b36c1eea2dc6ae5d6072
  Stored in directory: /root/.cache/pip/wheels/d0/bf/ac/2b3f43f8c6fcd0f4ba5395397458c521eb0b52d33b574a5a40
Successfully built paho-mqtt
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
ERROR: pip's dependency resolver does not currently take in

In [ ]:
CONNECTION_STRING = "HostName=Telemetria2022.azure-devices.net;DeviceId=myDeviceId;SharedAccessKey=I1Cs+Bbob8DPs/NX6OUaKkiPwSVGRtfBibmOQlF7JTw=";

# Define the JSON message to send to IoT Hub.
TEMPERATURE = 20.0
HUMIDITY = 60
MSG_TXT = '{{"velocidade": {velocidade},"potencia": {potencia}}}'
MSG_TXT = '{{"dados": {dados}}}'


INTERVAL = 1


def create_client():
    # Create an IoT Hub client
    client = IoTHubDeviceClient.create_from_connection_string(CONNECTION_STRING)

    # Define a method request handler
    def method_request_handler(method_request):
        if method_request.name == "SetTelemetryInterval":
            try:
                global INTERVAL
                INTERVAL = int(method_request.payload)
            except ValueError:
                response_payload = {"Response": "Invalid parameter"}
                response_status = 400
            else:
                response_payload = {"Response": "Executed direct method {}".format(method_request.name)}
                response_status = 200
        else:
            response_payload = {"Response": "Direct method {} not defined".format(method_request.name)}
            response_status = 404

        method_response = MethodResponse.create_from_method_request(method_request, response_status, response_payload)
        client.send_method_response(method_response)

    try:
        # Attach the method request handler
        client.on_method_request_received = method_request_handler
    except:
        # Clean up in the event of failure
        client.shutdown()
        raise

    return client


def run_telemetry_sample(client):
    # This sample will send temperature telemetry every second
    print("IoT Hub device sending periodic messages")

    client.connect()

    while True:
        # Build the message with simulated telemetry values.
        velocidade = TEMPERATURE + (random.random() * 15)
        potencia = HUMIDITY + (random.random() * 20)
        msg_txt_formatted = MSG_TXT.format(velocidade=velocidade, potencia=potencia)
        message = Message(msg_txt_formatted)

        # Add a custom application property to the message.
        # An IoT hub can filter on these properties without access to the message body.
        # Send the message.
        print("Sending message: {}".format(message))
        client.send_message(message)
        print("Message sent")
        time.sleep(INTERVAL)


def main():
    print ("IoT Hub Quickstart #2 - Simulated device")
    print ("Press Ctrl-C to exit")

    # Instantiate the client. Use the same instance of the client for the duration of
    # your application
    client = create_client()

    # Send telemetry
    try:
        run_telemetry_sample(client)
    except KeyboardInterrupt:
        print("IoTHubClient sample stopped by user")
    finally:
        print("Shutting down IoTHubClient")
        client.shutdown()


if __name__ == '__main__':
    main()

IoT Hub Quickstart #2 - Simulated device
Press Ctrl-C to exit
IoT Hub device sending periodic messages
Sending message: {"velocidade": 27.256042724782354,"potencia": 75.71744351865489}
Message sent
Sending message: {"velocidade": 21.70494548884486,"potencia": 61.48888584350916}
Message sent
Sending message: {"velocidade": 23.967218730069057,"potencia": 64.79929072596123}
Message sent
Sending message: {"velocidade": 22.609101405126392,"potencia": 67.71900763367785}
Message sent
